In [3]:
sc

res5: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4de91056

In [4]:
val in = scala.io.Source.fromURL("http://172.31.2.11:31410/db/charmander/series?u=root&p=root&q="+java.net.URLEncoder.encode(" select memory_usage from stats limit 200"), "utf-8")

in: scala.io.BufferedSource = non-empty iterator

In [5]:
var data = ""
for (line <- in.getLines)
data = line

data: String = [{"name":"stats","columns":["time","sequence_number","memory_usage"],"points":[[1419995633000,111600001,84946944],[1419995633000,111000001,84946944],[1419995633000,110400001,84946944],[1419995632000,111590001,84946944],[1419995632000,110990001,84946944],[1419995632000,110390001,84946944],[1419995631000,111580001,84946944],[1419995631000,110980001,84946944],[1419995631000,110380001,84946944],[1419995630000,111570001,84946944],[1419995630000,110970001,84946944],[1419995630000,110370001,84946944],[1419995629000,111560001,84946944],[1419995629000,110960001,84946944],[1419995629000,110360001,84946944],[1419995628000,111550001,84946944],[1419995628000,110950001,84946944],[1419995628000,110350001,84946944],[1419995627000,111540001,84946944],[1419995627000,110940001,84946944],[14...

In [6]:
import org.json4s.jackson.JsonMethods


import org.json4s.jackson.JsonMethods

In [7]:
val json = JsonMethods.parse(data)

json: org.json4s.JValue = JArray(List(JObject(List((name,JString(stats)), (columns,JArray(List(JString(time), JString(sequence_number), JString(memory_usage)))), (points,JArray(List(JArray(List(JInt(1419995633000), JInt(111600001), JInt(84946944))), JArray(List(JInt(1419995633000), JInt(111000001), JInt(84946944))), JArray(List(JInt(1419995633000), JInt(110400001), JInt(84946944))), JArray(List(JInt(1419995632000), JInt(111590001), JInt(84946944))), JArray(List(JInt(1419995632000), JInt(110990001), JInt(84946944))), JArray(List(JInt(1419995632000), JInt(110390001), JInt(84946944))), JArray(List(JInt(1419995631000), JInt(111580001), JInt(84946944))), JArray(List(JInt(1419995631000), JInt(110980001), JInt(84946944))), JArray(List(JInt(1419995631000), JInt(110380001), JInt(84946944))), JAr...

In [8]:
val points = json \\ "points"

points: org.json4s.JValue = JArray(List(JArray(List(JInt(1419995633000), JInt(111600001), JInt(84946944))), JArray(List(JInt(1419995633000), JInt(111000001), JInt(84946944))), JArray(List(JInt(1419995633000), JInt(110400001), JInt(84946944))), JArray(List(JInt(1419995632000), JInt(111590001), JInt(84946944))), JArray(List(JInt(1419995632000), JInt(110990001), JInt(84946944))), JArray(List(JInt(1419995632000), JInt(110390001), JInt(84946944))), JArray(List(JInt(1419995631000), JInt(111580001), JInt(84946944))), JArray(List(JInt(1419995631000), JInt(110980001), JInt(84946944))), JArray(List(JInt(1419995631000), JInt(110380001), JInt(84946944))), JArray(List(JInt(1419995630000), JInt(111570001), JInt(84946944))), JArray(List(JInt(1419995630000), JInt(110970001), JInt(84946944))), JArray(Li...

In [9]:
val mypoints = points.values

mypoints: points.Values = List(List(1419995633000, 111600001, 84946944), List(1419995633000, 111000001, 84946944), List(1419995633000, 110400001, 84946944), List(1419995632000, 111590001, 84946944), List(1419995632000, 110990001, 84946944), List(1419995632000, 110390001, 84946944), List(1419995631000, 111580001, 84946944), List(1419995631000, 110980001, 84946944), List(1419995631000, 110380001, 84946944), List(1419995630000, 111570001, 84946944), List(1419995630000, 110970001, 84946944), List(1419995630000, 110370001, 84946944), List(1419995629000, 111560001, 84946944), List(1419995629000, 110960001, 84946944), List(1419995629000, 110360001, 84946944), List(1419995628000, 111550001, 84946944), List(1419995628000, 110950001, 84946944), List(1419995628000, 110350001, 84946944), List(14199...

In [10]:
val rdd = sc.parallelize(mypoints.asInstanceOf[List[List[BigDecimal]]])

rdd: org.apache.spark.rdd.RDD[List[BigDecimal]] = ParallelCollectionRDD[1] at parallelize at <console>:55

In [11]:
case class MemoryUsage(timestamp: BigDecimal, memory: BigDecimal)

defined class MemoryUsage

In [12]:
val memoryusage = rdd.map(p => MemoryUsage(BigDecimal(p(0).asInstanceOf[BigInt]), BigDecimal(p(2).asInstanceOf[BigInt])))

memoryusage: org.apache.spark.rdd.RDD[MemoryUsage] = MappedRDD[2] at map at <console>:65

In [13]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext._

sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@31b84ec9
import sqlContext._

In [14]:
memoryusage.registerTempTable("memoryusage")

In [15]:
val newestMaxRaw = sqlContext.sql("select max(memory) from memoryusage").first()

newestMaxRaw: org.apache.spark.sql.Row = [84946944]